Data Retreival Process

In [1]:
import pandas as pd
import yfinance as yf
import time

def get_djia_tickers():
    """
    Returns a hardcoded list of DJIA tickers.
    Adjust these if the DJIA components change.
    """
    return [
        "MMM",  # 3M
        "AXP",  # American Express
        "AMGN", # Amgen
        "AAPL", # Apple
        "BA",   # Boeing
        "CAT",  # Caterpillar
        "CVX",  # Chevron
        "CSCO", # Cisco
        "KO",   # Coca-Cola
        "DIS",  # Disney
        "DOW",  # Dow Inc.
        "GS",   # Goldman Sachs
        "HD",   # Home Depot
        "HON",  # Honeywell
        "IBM",  # IBM
        "INTC", # Intel
        "JNJ",  # Johnson & Johnson
        "JPM",  # JPMorgan Chase
        "MCD",  # McDonald's
        "MRK",  # Merck
        "MSFT", # Microsoft
        "NKE",  # Nike
        "PG",   # Procter & Gamble
        "CRM",  # Salesforce
        "TRV",  # Travelers
        "UNH",  # UnitedHealth
        "VZ",   # Verizon
        "V",    # Visa
        "WBA",  # Walgreens Boots Alliance
        "WMT"   # Walmart
    ]

def save_tickers_to_csv(tickers, filename="DJIA_constituents.csv"):
    """
    Saves a list of tickers to a CSV file using pandas.
    """
    tickers_df = pd.DataFrame(tickers, columns=['Ticker'])
    tickers_df.to_csv(filename, index=False)

def fetch_history(tickers, period="10y"):
    """
    Fetches daily historical close prices for each ticker using yfinance.
    Returns a dictionary of DataFrames keyed by ticker symbol.
    """
    histories = {}
    for ticker in tickers:
        try:
            stock = yf.Ticker(ticker)
            history = stock.history(period=period)
            if not history.empty:
                # Keep only the 'Close' column for simplicity
                histories[ticker] = history["Close"]
            else:
                print(f"No data found for {ticker}")
        except Exception as e:
            print(f"Encountered error with stock symbol {ticker}: {e}")
        # Short pause to reduce risk of hitting rate limits
        time.sleep(0.5)
    return histories

def save_history_to_csv(histories, filename="DJIA_history.csv"):
    """
    Saves the historical close prices to a CSV file. Each column is a ticker.
    The index will be the date.
    """
    history_df = pd.DataFrame(histories)
    history_df.to_csv(filename, index=True)

def load_data_and_handle_missing(tickers_filename="DJIA_constituents.csv",
                                 history_filename="DJIA_history.csv"):
    """
    Loads the ticker symbols and historical data from CSVs into DataFrames.
    Applies forward fill to handle missing price data.
    """
    tickers_df = pd.read_csv(tickers_filename)
    history_df = pd.read_csv(history_filename, index_col=0)
    
    # Forward fill missing price data
    history_df.fillna(method='ffill', inplace=True)
    
    return tickers_df, history_df

def main():
    # 1. Define the tickers manually (hardcoded list).
    tickers = get_djia_tickers()

    # 2. Save tickers to CSV
    save_tickers_to_csv(tickers)

    # 3. Fetch 10 years of daily adjusted close prices
    histories = fetch_history(tickers, period="10y")

    # 4. Save the historical data to CSV
    save_history_to_csv(histories)

    # 5. Load the data from CSVs and handle missing data
    tickers_df, history_df = load_data_and_handle_missing()
    
    # 6. Display a preview of the loaded DataFrames
    print("Tickers DataFrame:")
    print(tickers_df.head())
    print("\nHistory DataFrame (first 5 rows):")
    print(history_df.head())

if __name__ == '__main__':
    main()

Tickers DataFrame:
  Ticker
0    MMM
1    AXP
2   AMGN
3   AAPL
4     BA

History DataFrame (first 5 rows):
                                  MMM        AXP        AMGN       AAPL  \
Date                                                                      
2015-03-04 00:00:00-05:00   99.901192  69.664619  119.369545  28.706535   
2015-03-05 00:00:00-05:00  100.146217  70.113922  119.677002  28.230854   
2015-03-06 00:00:00-05:00   98.227783  69.388145  116.137688  28.273281   
2015-03-09 00:00:00-04:00   99.429031  69.560951  116.235199  28.393883   
2015-03-10 00:00:00-04:00   96.972755  68.351349  115.162865  27.806528   

                                   BA        CAT        CVX       CSCO  \
Date                                                                     
2015-03-04 00:00:00-05:00  135.832977  62.876461  68.536324  21.446743   
2015-03-05 00:00:00-05:00  135.938553  62.707199  68.256081  21.424807   
2015-03-06 00:00:00-05:00  134.750534  61.591671  67.487030  21.154255

In [2]:
def calculate_daily_returns(history_df):
    """
    Calculate daily returns for each ticker based on historical close prices.
    """
    return history_df.pct_change()

if __name__ == '__main__':
    # Load the data as before
    tickers_df, history_df = load_data_and_handle_missing()
    
    # Calculate daily returns
    daily_returns_df = calculate_daily_returns(history_df)
    
    # Display a preview of the DataFrame
    print("\nDaily Returns DataFrame (first 5 rows):")
    print(daily_returns_df.head())


Daily Returns DataFrame (first 5 rows):
                                MMM       AXP      AMGN      AAPL        BA  \
Date                                                                          
2015-03-04 00:00:00-05:00       NaN       NaN       NaN       NaN       NaN   
2015-03-05 00:00:00-05:00  0.002453  0.006450  0.002576 -0.016570  0.000777   
2015-03-06 00:00:00-05:00 -0.019156 -0.010351 -0.029574  0.001503 -0.008739   
2015-03-09 00:00:00-04:00  0.012229  0.002490  0.000840  0.004266  0.010645   
2015-03-10 00:00:00-04:00 -0.024704 -0.017389 -0.009226 -0.020686 -0.015057   

                                CAT       CVX      CSCO        KO       DIS  \
Date                                                                          
2015-03-04 00:00:00-05:00       NaN       NaN       NaN       NaN       NaN   
2015-03-05 00:00:00-05:00 -0.002692 -0.004089 -0.001023 -0.002353 -0.005115   
2015-03-06 00:00:00-05:00 -0.017789 -0.011267 -0.012628 -0.020755 -0.011521   
2015-03-09

In [6]:
import pandas as pd

def main():
    # 1. Read the CSV (with wide-format close prices)
    df = pd.read_csv("DJIA_history.csv", parse_dates=["Date"])
    
    # 2. Set 'Date' as the DataFrame index
    df.set_index("Date", inplace=True)
    
    # 3. Compute daily returns and drop the first row (NaNs from no previous day)
    daily_returns = df.pct_change().iloc[1:]
    
    # 4. Convert daily returns to long (stacked) format
    #    Now we have a Series with a MultiIndex: (Date, Symbol)
    dr_stacked = daily_returns.stack()
    dr_stacked.name = "Daily Return"
    
    # 5. For each Date, pick the 10 smallest returns
    #    This yields a Series still indexed by (Date, Symbol), but only 10 tickers per Date
    lowest_10 = dr_stacked.groupby(level="Date", group_keys=False).apply(lambda x: x.nsmallest(10))
    
    # 6. Convert that Series to a DataFrame
    lowest_10_df = lowest_10.to_frame()
    
    # 7. Get the corresponding close prices in stacked format
    close_stacked = df.stack()
    close_stacked.name = "Close Price"
    
    # 8. Align (reindex) the close prices with our (Date, Symbol) pairs
    lowest_10_df["Close Price"] = close_stacked.reindex(lowest_10_df.index)
    
    # 9. Reset index so 'Date' and 'Symbol' become columns
    lowest_10_df.reset_index(inplace=True)
    lowest_10_df.rename(columns={"level_1": "Symbol"}, inplace=True)
    
    # 10. Reorder columns to match desired format
    lowest_10_df = lowest_10_df[["Date", "Symbol", "Close Price", "Daily Return"]]
    
    # 11. Save the result to a new CSV
    lowest_10_df.to_csv("lowest_returns_with_close.csv", index=False)
    print("lowest_returns_with_close.csv has been created.")

if __name__ == "__main__":
    main()

lowest_returns_with_close.csv has been created.


In [13]:
import pandas as pd

# Load the dataset
file_path = "/data/notebook_files/lowest_returns_with_close.csv"
df = pd.read_csv(file_path)

# Ensure correct column formatting
df.columns = df.columns.str.strip()

# Convert 'Date' column to datetime for sorting
df['Date'] = pd.to_datetime(df['Date'])

# Sort data by Date
df = df.sort_values(by=['Date'])

# Initial capital
capital = 100000

# Dictionary to track portfolio value over time
portfolio_history = {}

# Unique dates
unique_dates = df['Date'].unique()

# Simulating the trading strategy
for i in range(len(unique_dates) - 1):  # Exclude the last date since we can't sell the next day
    today = unique_dates[i]
    tomorrow = unique_dates[i + 1]

    # Select today's top 10 biggest losers
    today_data = df[df['Date'] == today].nsmallest(10, 'Daily Return')

    if today_data.empty:
        continue

    # Allocate equal capital to each stock
    allocation = capital / 10

    # Store purchased stocks and their prices
    positions = {}
    for _, row in today_data.iterrows():
        symbol = row['Symbol']
        close_price = row['Close Price']
        shares = allocation / close_price if close_price > 0 else 0  # Avoid division by zero
        positions[symbol] = shares

    # Move to the next day and sell holdings
    tomorrow_data = df[df['Date'] == tomorrow]

    daily_return = 0
    for symbol, shares in positions.items():
        sell_price = tomorrow_data[tomorrow_data['Symbol'] == symbol]['Close Price']
        if not sell_price.empty:
            daily_return += shares * sell_price.values[0]

    # Update capital
    capital = daily_return

    # Store portfolio value
    portfolio_history[tomorrow] = capital

# Convert to DataFrame and save results
portfolio_df = pd.DataFrame(list(portfolio_history.items()), columns=['Date', 'Portfolio Value'])
portfolio_df.to_csv("/data/notebook_files/trading_simulation_results.csv", index=False)

print("Simulation complete. Results saved to 'trading_simulation_results.csv'.")

Simulation complete. Results saved to 'trading_simulation_results.csv'.


In [1]:
import pandas as pd
import numpy as np

def simulate_biggest_losers_strategy_hold(
    csv_path,
    initial_capital=100000.0,
    output_path="trading_simulation_results_hold.csv"
):
    """
    Advanced: Buy the 10 biggest losers each day, hold for at least 1 day,
    and if a symbol is missing on the next day, keep holding until it reappears.
    
    CSV must have columns: Date, Symbol, Close Price, Daily Return
    """

    # 1. Read the CSV
    df = pd.read_csv(csv_path)
    df.columns = df.columns.str.strip()

    # Required columns
    required_cols = {"Date", "Symbol", "Close Price", "Daily Return"}
    if not required_cols.issubset(df.columns):
        raise ValueError(f"CSV must contain columns named exactly: {required_cols}")

    # 2. Convert 'Date' to datetime & sort
    df["Date"] = pd.to_datetime(df["Date"])
    df.sort_values("Date", inplace=True)

    # 3. Get unique dates in ascending order
    dates = df["Date"].unique()

    # 4. We'll keep a dictionary of open positions:
    #    open_positions[symbol] = {
    #        "buy_date_idx": index of the day we bought it,
    #        "last_known_price": last close we have for that symbol,
    #        "shares": how many shares we hold
    #    }
    open_positions = {}

    # 5. Track free capital and daily portfolio value
    capital = initial_capital
    portfolio_history = {}  # date -> portfolio value

    # 6. Map each (date, symbol) -> close price for quick lookups
    #    This helps us find the close price if it reappears
    price_lookup = {}
    for _, row in df.iterrows():
        d = row["Date"]
        sym = row["Symbol"]
        c = row["Close Price"]
        price_lookup.setdefault(d, {})[sym] = c

    # 7. Helper function to compute the total portfolio value on a given day
    #    using the last known prices of open positions plus free capital
    def compute_portfolio_value(day_idx):
        val = capital
        day = dates[day_idx]
        # For each open position, if we have a price for 'day', update it; else keep last known
        for sym, pos in open_positions.items():
            # If the symbol appears on 'day', update last_known_price
            if sym in price_lookup.get(day, {}):
                pos["last_known_price"] = price_lookup[day][sym]
            # Add to portfolio value
            val += pos["shares"] * pos["last_known_price"]
        return val

    # 8. Main simulation loop: day by day
    for i in range(len(dates)):
        today = dates[i]

        # Step A: Attempt to SELL any open positions that were bought at least 1 day ago
        #         IF the symbol appears in today's data
        to_remove = []
        for sym, pos in open_positions.items():
            buy_date_idx = pos["buy_date_idx"]
            # We only sell if we've held it at least 1 day
            if i > buy_date_idx:
                # If today's data has a price for that symbol, we can sell
                if sym in price_lookup.get(today, {}):
                    sell_price = price_lookup[today][sym]
                    proceeds = pos["shares"] * sell_price
                    capital += proceeds
                    to_remove.append(sym)
        # Remove sold positions from open_positions
        for sym in to_remove:
            del open_positions[sym]

        # Step B: If this is NOT the last date, BUY today's 10 biggest losers
        #         using whatever free capital we have left
        if i < len(dates) - 1:
            # Find the 10 biggest losers for 'today'
            # (We look up the rows in df for this exact date)
            today_data = df[df["Date"] == today]
            losers_today = today_data.nsmallest(10, "Daily Return")
            if not losers_today.empty:
                num_stocks = len(losers_today)
                if num_stocks > 0:
                    allocation_per_stock = capital / num_stocks
                else:
                    allocation_per_stock = 0

                # Buy each of these 'losers' at today's close
                for _, row in losers_today.iterrows():
                    sym = row["Symbol"]
                    close_price = row["Close Price"]
                    if close_price > 0:
                        shares = allocation_per_stock / close_price
                        # Deduct from capital
                        capital -= shares * close_price
                        # Store in open_positions
                        open_positions[sym] = {
                            "buy_date_idx": i,
                            "last_known_price": close_price,
                            "shares": shares
                        }

        # Step C: Record the portfolio value at the end of 'today'
        portfolio_value = compute_portfolio_value(i)
        portfolio_history[today] = portfolio_value

    # 9. Convert the portfolio history to a DataFrame and save
    results_df = pd.DataFrame({
        "Date": list(portfolio_history.keys()),
        "Portfolio Value": list(portfolio_history.values())
    })
    results_df.sort_values("Date", inplace=True)
    results_df.to_csv(output_path, index=False)

    final_day = dates[-1]
    final_value = portfolio_history[final_day]
    print(f"Simulation complete. Final capital (mark-to-market): {final_value:,.2f}")
    print(f"Results saved to {output_path}.")

In [3]:
simulate_biggest_losers_strategy_hold(
    csv_path="/data/notebook_files/lowest_returns_with_close.csv",  # <-- Adjust path as needed
    initial_capital=100000.0,  # or whatever starting capital you want
    output_path="/data/notebook_files/trading_simulation_results.csv"  # Where to save output
)

Simulation complete. Final capital (mark-to-market): 324,837.00
Results saved to /data/notebook_files/trading_simulation_results.csv.


In [1]:
import pandas as pd
import numpy as np

def calculate_performance_metrics(
    csv_path="trading_simulation_results.csv",
    date_col="Date",
    value_col="Portfolio Value",
    trading_days_per_year=252
):
    """
    Reads the CSV containing simulated portfolio values over time, then calculates:
    1) Annualized Return
    2) Annualized Volatility
    3) Sharpe Ratio (0% risk-free)
    4) Maximum Drawdown

    Args:
        csv_path (str): Path to the CSV file with 'Date' and 'Portfolio Value'.
        date_col (str): Name of the column representing dates.
        value_col (str): Name of the column representing portfolio values.
        trading_days_per_year (int): Typically 252 for equities.
    """

    # 1. Load the portfolio data
    df = pd.read_csv(csv_path)
    # Convert Date to datetime (if not already)
    df[date_col] = pd.to_datetime(df[date_col])
    df.sort_values(by=date_col, inplace=True)
    df.reset_index(drop=True, inplace=True)

    # 2. Calculate daily returns
    #    daily_return(t) = (value(t) - value(t-1)) / value(t-1)
    df["Daily Return"] = df[value_col].pct_change()

    # Remove the first row (NaN daily return)
    df.dropna(subset=["Daily Return"], inplace=True)

    # 3. Annualized Return
    #    Method A: (FinalValue / InitialValue)^(trading_days_per_year / total_days) - 1
    #    Method B: Use product of (1 + daily_return).
    total_days = df.shape[0]
    cumulative_return = (1 + df["Daily Return"]).prod() - 1
    # Geometric annualized return
    annualized_return = (1 + cumulative_return) ** (trading_days_per_year / total_days) - 1

    # 4. Annualized Volatility
    #    = std(daily_return) * sqrt(trading_days_per_year)
    daily_std = df["Daily Return"].std()
    annualized_volatility = daily_std * np.sqrt(trading_days_per_year)

    # 5. Sharpe Ratio (risk-free rate = 0)
    #    = (annualized_return - 0) / annualized_volatility
    if annualized_volatility == 0:
        sharpe_ratio = np.nan
    else:
        sharpe_ratio = annualized_return / annualized_volatility

    # 6. Maximum Drawdown
    #    Drawdown(t) = (PortfolioValue(t) / max(PortfolioValue up to t)) - 1
    df["Cumulative Peak"] = df[value_col].cummax()
    df["Drawdown"] = df[value_col] / df["Cumulative Peak"] - 1
    max_drawdown = df["Drawdown"].min()

    # 7. Print metrics
    print("Performance Metrics:")
    print(f"  Annualized Return:    {annualized_return:.2%}")
    print(f"  Annualized Volatility:{annualized_volatility:.2%}")
    print(f"  Sharpe Ratio:         {sharpe_ratio:.3f}")
    print(f"  Maximum Drawdown:     {max_drawdown:.2%}")

    return {
        "annualized_return": annualized_return,
        "annualized_volatility": annualized_volatility,
        "sharpe_ratio": sharpe_ratio,
        "max_drawdown": max_drawdown
    }

# ---------------------------
# Example usage:
# metrics = calculate_performance_metrics(
#     csv_path="/data/notebook_files/trading_simulation_results.csv",
#     date_col="Date",
#     value_col="Portfolio Value"
# )
# ---------------------------

In [2]:
metrics = calculate_performance_metrics(
    csv_path="/data/notebook_files/trading_simulation_results.csv",
    date_col="Date",
    value_col="Portfolio Value"
)

Performance Metrics:
  Annualized Return:    12.54%
  Annualized Volatility:9.17%
  Sharpe Ratio:         1.368
  Maximum Drawdown:     -32.55%


In [46]:
import pandas as pd
import numpy as np
import yfinance as yf

def compare_sharpe_ratios(
    strategy_csv="/data/notebook_files/trading_simulation_results.csv",
    date_col="Date",
    port_val_col="Portfolio Value",
    ticker="DIA"
):
    """
    Compares the Sharpe ratio of our mean reversion strategy (Portfolio Value) with that of DIA,
    flattening the multi-level columns returned by yfinance and using only the 'Close_DIA' column.

    Steps:
    1) Load 'Date' and 'Portfolio Value' from 'trading_simulation_results.csv'.
    2) Download DIA data for the same date range (multi-index columns -> flatten them).
    3) Use 'Close_DIA' to compute daily returns for DIA.
    4) Compare Sharpe ratios (risk-free = 0).
    """

    # --- PART A: Load Strategy Data ---
    df_strat = pd.read_csv(strategy_csv)

    # Ensure required columns exist
    if not {date_col, port_val_col}.issubset(df_strat.columns):
        raise ValueError(f"CSV must contain columns: '{date_col}' and '{port_val_col}'")

    # Convert date and sort
    df_strat[date_col] = pd.to_datetime(df_strat[date_col], errors="coerce")
    df_strat.sort_values(by=date_col, inplace=True)
    df_strat.reset_index(drop=True, inplace=True)

    # Create daily returns for your strategy
    df_strat["Daily Return"] = df_strat[port_val_col].pct_change()
    # Drop the first row with NaN daily return
    df_strat.dropna(subset=["Daily Return"], inplace=True)

    # Identify start & end dates
    start_date = df_strat[date_col].min()
    end_date = df_strat[date_col].max()

    # print("DEBUG: Strategy data head:\n", df_strat.head(), "\n")
    # print(f"DEBUG: Strategy start date = {start_date}, end date = {end_date}")

    # --- PART B: Download DIA data ---
    dia_data = yf.download(ticker, start=start_date, end=end_date, progress=False)
    # print("\nDEBUG: DIA raw data shape:", dia_data.shape)
    # print("DEBUG: DIA raw data columns (possibly multi-level):", dia_data.columns.tolist())
    # print(dia_data.head(), "\n")

    # If no rows returned
    if dia_data.empty:
        raise ValueError(f"No DIA data returned from {start_date} to {end_date}.")

    # --- PART C: Flatten multi-level columns ---
    if isinstance(dia_data.columns, pd.MultiIndex):
        # Convert e.g. ("Close","DIA") -> "Close_DIA"
        dia_data.columns = ["_".join(col).strip() for col in dia_data.columns.values]
    else:
        # Already single-level columns, do nothing
        dia_data.columns = dia_data.columns.astype(str)

    # print("DEBUG: DIA columns after flattening:", dia_data.columns.tolist())
    # print(dia_data.head(), "\n")

    # We expect a "Close_DIA" column if flattening occurred
    close_col = f"Close_{ticker}"
    if close_col not in dia_data.columns:
        # If we can't find "Close_DIA", check if there's a plain "Close"
        if "Close" in dia_data.columns:
            close_col = "Close"
        else:
            raise KeyError(
                f"Cannot find '{close_col}' or 'Close' in DIA data columns: {dia_data.columns.tolist()}"
            )

    # --- PART D: Compute DIA daily returns from close_col ---
    dia_data["Daily Return"] = dia_data[close_col].pct_change()
    # Drop the first row with NaN daily return
    dia_data.dropna(subset=["Daily Return"], inplace=True)

    # print("DEBUG: DIA data after creating 'Daily Return':\n", dia_data.head(), "\n")

    # --- PART E: Sharpe Ratio Function ---
    def annualized_sharpe(daily_returns, days_per_year=252):
        daily_returns = daily_returns.dropna()
        if len(daily_returns) == 0:
            return np.nan
        mean_ret = daily_returns.mean()
        std_ret = daily_returns.std()
        if std_ret == 0:
            return np.nan
        return mean_ret / std_ret * np.sqrt(days_per_year)

    # --- PART F: Calculate Sharpe Ratios ---
    strat_sharpe = annualized_sharpe(df_strat["Daily Return"])
    dia_sharpe = annualized_sharpe(dia_data["Daily Return"])

    # --- PART G: Print Results ---
    print(f"\nStrategy Sharpe Ratio: {strat_sharpe:.3f}")
    print(f"DIA Sharpe Ratio:      {dia_sharpe:.3f}")

    if pd.isna(strat_sharpe) or pd.isna(dia_sharpe):
        print("Warning: One of the Sharpe Ratios is NaN (insufficient data).")
    else:
        if strat_sharpe > dia_sharpe:
            print("Our strategy outperformed DIA based on Sharpe ratio!")
        elif strat_sharpe < dia_sharpe:
            print("Our strategy underperformed DIA based on Sharpe ratio.")
        else:
            print("Our strategy's Sharpe ratio is exactly the same as DIA's.")

In [47]:
compare_sharpe_ratios(
    strategy_csv="/data/notebook_files/trading_simulation_results.csv",
    date_col="Date",
    port_val_col="Portfolio Value",
    ticker="DIA"
)


Strategy Sharpe Ratio: 1.335
DIA Sharpe Ratio:      0.724
Our strategy outperformed DIA based on Sharpe ratio!


In [49]:
import pandas as pd
import numpy as np
import yfinance as yf

# 1) Define a helper function to compute annual metrics.
def compute_metrics(daily_returns, trading_days=252):
    """
    Computes annual return, annual standard deviation, and Sharpe ratio (RF=0)
    given a Series of daily returns.
    
    Returns:
      (annual_return, annual_std, sharpe_ratio)
    """
    daily_returns = daily_returns.dropna()
    if len(daily_returns) < 2:
        return np.nan, np.nan, np.nan
    
    # Total cumulative return
    cumulative_return = (1 + daily_returns).prod() - 1
    
    # Annualized Return = (1 + cumulative_return)^(252 / N) - 1
    num_days = len(daily_returns)
    annual_return = (1 + cumulative_return) ** (trading_days / num_days) - 1
    
    # Annualized Volatility = std(daily_returns) * sqrt(252)
    annual_std = daily_returns.std() * np.sqrt(trading_days)
    
    # Sharpe Ratio (risk-free = 0)
    if annual_std == 0:
        sharpe_ratio = np.nan
    else:
        sharpe_ratio = annual_return / annual_std
    
    return annual_return, annual_std, sharpe_ratio

# 2) Load your strategy’s results
strategy_csv = "/data/notebook_files/trading_simulation_results.csv"
df_strat = pd.read_csv(strategy_csv)

# Ensure it has "Date" and "Portfolio Value"
df_strat["Date"] = pd.to_datetime(df_strat["Date"], errors="coerce")
df_strat.sort_values("Date", inplace=True)
df_strat.reset_index(drop=True, inplace=True)

# Create daily returns
df_strat["Daily Return"] = df_strat["Portfolio Value"].pct_change()
df_strat.dropna(subset=["Daily Return"], inplace=True)

# 3) Fetch DIA data for the same date range
start_date = df_strat["Date"].min()
end_date = df_strat["Date"].max()

df_dia = yf.download("DIA", start=start_date, end=end_date, progress=False)

# If yfinance returns multi-level columns, flatten them
if isinstance(df_dia.columns, pd.MultiIndex):
    df_dia.columns = ["_".join(col).strip() for col in df_dia.columns.values]

# Use "Close_DIA" if it exists, else just "Close"
if "Close_DIA" in df_dia.columns:
    close_col = "Close_DIA"
elif "Close" in df_dia.columns:
    close_col = "Close"
else:
    raise KeyError(f"No 'Close' column found in DIA data columns: {df_dia.columns.tolist()}")

# Compute DIA daily returns
df_dia["Daily Return"] = df_dia[close_col].pct_change()
df_dia.dropna(subset=["Daily Return"], inplace=True)

# 4) Compute metrics for both
strat_annual_return, strat_annual_vol, strat_sharpe = compute_metrics(df_strat["Daily Return"])
dia_annual_return, dia_annual_vol, dia_sharpe = compute_metrics(df_dia["Daily Return"])

# 5) Print them side by side
print("Mean Reversion Strategy vs. DIA Performance Metrics")
print("                  Annual Return   Annual StdDev   Sharpe Ratio")
print(f"Strategy (MRS):   {strat_annual_return:7.2%}       {strat_annual_vol:7.2%}       {strat_sharpe:7.3f}")
print(f"DIA:              {dia_annual_return:7.2%}       {dia_annual_vol:7.2%}       {dia_sharpe:7.3f}")

# Indicate which strategy had higher risk-adjusted returns
if pd.isna(strat_sharpe) or pd.isna(dia_sharpe):
    print("\nWarning: One or both Sharpe ratios is NaN (insufficient data).")
elif strat_sharpe > dia_sharpe:
    print("\nOur mean reversion strategy had higher risk-adjusted returns!")
elif strat_sharpe < dia_sharpe:
    print("\nDIA had higher risk-adjusted returns.")
else:
    print("\nBoth have the same Sharpe ratio!")

Mean Reversion Strategy vs. DIA Performance Metrics
                  Annual Return   Annual StdDev   Sharpe Ratio
Strategy (MRS):    12.54%         9.17%         1.368
DIA:               11.73%        17.43%         0.673

Our mean reversion strategy had higher risk-adjusted returns!


In [52]:
import pandas as pd
import plotly.graph_objs as go
import yfinance as yf
import numpy as np

def load_strategy_data(csv_path):
    """
    Loads the strategy CSV, which must have:
      - 'Date'
      - 'Portfolio Value'
    Then computes the 'Daily Return' from 'Portfolio Value'.
    Returns a DataFrame with columns: ['Date', 'Portfolio Value', 'Daily Return'].
    """
    df = pd.read_csv(csv_path)
    
    # 1) Check columns
    required_cols = {"Date", "Portfolio Value"}
    if not required_cols.issubset(df.columns):
        raise ValueError(f"CSV must have columns: {required_cols}")
    
    # 2) Convert 'Date' to datetime, sort
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    df.sort_values("Date", inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    # 3) Create 'Daily Return' from 'Portfolio Value'
    df["Daily Return"] = df["Portfolio Value"].pct_change()
    df.dropna(subset=["Daily Return"], inplace=True)
    
    return df

def load_dia_data(start_date, end_date):
    """
    Fetches DIA data from Yahoo Finance between start_date and end_date.
    Ensures we have a single-level column for 'Close' or 'Close_DIA'.
    Computes 'Daily Return' from the close column.
    Returns a DataFrame with columns: ['Date', 'Close', 'Daily Return'].
    """
    dia_data = yf.download("DIA", start=start_date, end=end_date, progress=False)
    
    # If multi-level columns, flatten them
    if isinstance(dia_data.columns, pd.MultiIndex):
        dia_data.columns = ["_".join(col).strip() for col in dia_data.columns.values]
    
    # Decide which column to use for close
    close_col = "Close_DIA" if "Close_DIA" in dia_data.columns else "Close"
    if close_col not in dia_data.columns:
        raise KeyError(f"No 'Close' column found in DIA data columns: {dia_data.columns.tolist()}")
    
    # Convert index to 'Date' column
    dia_data.reset_index(inplace=True)
    dia_data.rename(columns={"Date": "Date"}, inplace=True)
    dia_data.sort_values("Date", inplace=True)
    
    # Compute daily returns
    dia_data["Daily Return"] = dia_data[close_col].pct_change()
    dia_data.dropna(subset=["Daily Return"], inplace=True)
    
    # Keep only relevant columns
    return dia_data[["Date", close_col, "Daily Return"]].rename(columns={close_col: "Close"})

def plot_growth(df_strat, df_dia, initial_investment=100000.0):
    """
    1) Calculate cumulative returns for both df_strat and df_dia using 'Daily Return'.
    2) Multiply by initial_investment to get daily portfolio values.
    3) Plot an interactive line chart with Plotly.
    """
    # 1) Calculate cumulative returns
    df_strat["Cumulative Return"] = (1 + df_strat["Daily Return"]).cumprod()
    df_dia["Cumulative Return"]   = (1 + df_dia["Daily Return"]).cumprod()
    
    # 2) Multiply by initial_investment
    df_strat["Portfolio Growth"] = df_strat["Cumulative Return"] * initial_investment
    df_dia["Portfolio Growth"]   = df_dia["Cumulative Return"]   * initial_investment
    
    # 3) Create Plotly figure
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=df_strat["Date"],
        y=df_strat["Portfolio Growth"],
        mode="lines",
        name="Mean Reversion Strategy"
    ))
    
    fig.add_trace(go.Scatter(
        x=df_dia["Date"],
        y=df_dia["Portfolio Growth"],
        mode="lines",
        name="DIA ETF"
    ))
    
    fig.update_layout(
        title="Growth of $100,000: Mean Reversion Strategy vs. DIA",
        xaxis_title="Date",
        yaxis_title="Portfolio Value (USD)",
        legend=dict(x=0.02, y=0.95),
        hovermode="x unified",
        width=1200,  
        height=800 
    )
    
    fig.show()

# ---------------------------
# Example usage:
# 1) Load your strategy data
df_strat = load_strategy_data("/data/notebook_files/trading_simulation_results.csv")

# 2) Get date range
start_date = df_strat["Date"].min()
end_date   = df_strat["Date"].max()

# 3) Load DIA data from Yahoo Finance
df_dia = load_dia_data(start_date, end_date)

# 4) Plot
plot_growth(df_strat, df_dia, initial_investment=100000.0)
# ---------------------------